In [1]:
import polars as pl
import pandas as pd
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

In [2]:
data_path = '/home/gyli/GP_HP/data/hypertension.csv'
raw_data = pl.read_csv(data_path, dtypes={'年龄' : pl.Float64}).with_columns(pl.col('年龄').cast(pl.Int64))

In [3]:
raw_data.columns

['年龄',
 '中性粒细胞数',
 '淋巴细胞数',
 '嗜酸性粒细胞数',
 '嗜碱性粒细胞数',
 '总蛋白',
 '白蛋白',
 '球蛋白',
 '总胆红素',
 '直接胆红素',
 '钾',
 '钠',
 '钙',
 '尿素氮',
 '谷草谷丙',
 '甘油三酯',
 '高密度脂蛋白',
 '低密度脂蛋白',
 '红细胞平均体积',
 '凝血酶原时间',
 '国际标准化比值',
 '活化部分凝血活酶时间',
 '纤维蛋白原',
 '是否高血压']

In [5]:
raw_data.describe().write_csv('ttt.csv')

In [14]:
raw_data.shape

(33287, 24)

In [18]:
raw_data.filter(pl.col('年龄') < 1)

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原,是否高血压
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
0,4.96,2.51,0.06,0.0,62.2,37.9,24.3,8.7,2.7,3.961723,140.880901,2.226504,4.564519,1.5,0.98,1.68,2.9,98.0,12.7,0.98,37.7,4.64,0
0,4.885651,2.247269,0.1767,0.014171,75.8,44.9,30.9,10.5,2.9,3.8,142.0,2.49,5.3,1.2,1.18,1.41,3.3,92.087414,12.805008,1.001135,33.431035,3.918851,0
0,18.57,4.33,0.2,0.12,50.3,31.7,18.6,96.2,6.2,5.1,141.7,1.96,8.5,6.4,1.779246,1.87046,2.771729,107.2,12.949614,0.990166,34.685492,4.949974,0
0,5.02,2.42,0.15,0.0,44.6,18.4,26.2,5.3,0.8,3.8,140.0,2.85,9.8,1.5,1.315927,1.416631,2.86093,96.8,12.720712,0.998678,32.345378,3.394177,0
0,5.78,2.33,0.05,0.0,41.7,12.5,29.2,3.1,0.7,3.6,141.0,2.02,5.5,1.9,1.325564,1.461032,2.854548,93.5,12.531642,0.987379,31.730008,3.509742,0
0,6.48,2.74,0.25,0.02,54.771741,30.349769,24.42826,7.89242,2.291928,4.4,141.0,1.68,1.0,1.177081,1.888296,1.376351,2.909613,107.2,13.187249,1.017804,34.744938,3.725706,0
0,20.01,5.32,0.28,0.08,60.929035,33.388982,27.543561,9.9005,2.65883,3.751221,138.314364,2.17827,1.4,1.302109,1.366557,1.534254,2.844574,104.6,12.2,0.89,41.9,3.53,0
0,8.3,2.06,0.55,0.01,60.033584,33.299721,26.736938,7.098926,2.073033,4.0,140.0,2.0,1.0,1.170647,1.90308,1.363987,2.915877,104.7,13.111966,1.014731,34.448263,3.713614,0
0,5.14,8.15,0.24,0.03,76.447044,50.979759,25.468512,14.165689,3.214224,4.013859,143.869074,2.449987,4.514982,0.927715,1.028259,1.18794,2.274966,123.8,19.5,1.71,63.7,1.51,0


In [26]:
test = []
for c in raw_data.columns:
        test.append((c, raw_data.filter(pl.col(c) == 0)))

In [27]:
test

[('年龄',
  shape: (26, 24)
  ┌──────┬────────────┬────────────┬────────────┬───┬────────────┬───────────┬───────────┬───────────┐
  │ 年龄 ┆ 中性粒细胞 ┆ 淋巴细胞数 ┆ 嗜酸性粒细 ┆ … ┆ 国际标准化 ┆ 活化部分  ┆ 纤维蛋白  ┆ 是否高血  │
  │ ---  ┆ 数         ┆ ---        ┆ 胞数       ┆   ┆ 比值       ┆ 凝血活酶  ┆ 原        ┆ 压        │
  │ i64  ┆ ---        ┆ f64        ┆ ---        ┆   ┆ ---        ┆ 时间      ┆ ---       ┆ ---       │
  │      ┆ f64        ┆            ┆ f64        ┆   ┆ f64        ┆ ---       ┆ f64       ┆ i64       │
  │      ┆            ┆            ┆            ┆   ┆            ┆ f64       ┆           ┆           │
  ╞══════╪════════════╪════════════╪════════════╪═══╪════════════╪═══════════╪═══════════╪═══════════╡
  │ 0    ┆ 4.96       ┆ 2.51       ┆ 0.06       ┆ … ┆ 0.98       ┆ 37.7      ┆ 4.64      ┆ 0         │
  │ 0    ┆ 4.885651   ┆ 2.247269   ┆ 0.1767     ┆ … ┆ 1.001135   ┆ 33.431035 ┆ 3.918851  ┆ 0         │
  │ 0    ┆ 18.57      ┆ 4.33       ┆ 0.2        ┆ … ┆ 0.990166   ┆ 34.685492 ┆ 4.949974  ┆ 0  

In [32]:
test[4][1]

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原,是否高血压
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
33,2.5,1.56,0.08,0.0,72.7,49.2,23.5,17.2,3.7,3.3,141.0,2.17,3.9,0.7,1.36,1.26,3.49,92.1,13.0,1.01,36.7,2.94,0
71,2.95,1.95,0.09,0.0,65.1,39.5,25.6,6.1,1.3,4.2,144.0,2.07,3.6,1.2,1.505878,1.453503,2.966601,90.0,12.5,0.96,31.3,4.15,0
83,7.87,1.88,0.11,0.0,74.5,43.4,31.1,9.9,2.4,4.3,141.0,2.27,4.5,0.9,2.354986,1.295662,3.051663,96.8,12.5,0.92,37.2,4.2,1
50,4.25,1.98,0.43,0.0,65.6,41.9,23.7,10.4,3.0,4.1,141.0,2.3,4.5,0.7,2.36,0.83,2.11,95.0,11.7,0.85,32.9,3.94,1
61,2.36,2.6,0.11,0.0,65.0,39.1,25.9,5.8,2.1,3.7,144.0,2.22,4.5,1.3,0.83,1.2,2.81,89.5,12.647543,1.009092,32.12749,3.746655,0
60,4.49,1.9,0.09,0.0,68.7,42.1,26.6,9.1,2.4,4.044071,141.309462,2.293508,5.126985,1.0,1.36,1.28,3.14,95.2,12.5,0.96,32.3,2.91,0
53,5.22,2.34,0.36,0.0,68.231833,42.042988,26.189187,10.397208,2.798564,4.0687359,141.439245,2.291009,5.674648,0.917544,3.54,0.6,2.16,96.6,12.7,0.98,36.2,3.72,1
54,3.27,1.95,0.03,0.0,77.3,49.8,27.5,8.3,2.2,3.9,142.0,2.5,4.5,1.2,1.51,1.29,3.44,93.5,12.519405,0.986291,32.008736,3.582925,1
65,3.18,2.01,0.44,0.0,63.2,41.1,22.1,11.6,2.4,3.6,142.0,2.17,8.8,0.9,1.714552,1.209259,2.812669,95.9,12.5,0.96,32.9,2.1,0


In [33]:
raw_data

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原,是否高血压
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64
79,9.26,2.14,0.18,0.03,67.948927,39.546489,28.401192,10.637813,3.191754,4.1,137.2,2.34,5.2,1.2,0.57,1.1,1.67,95.0,13.323282,1.06193,33.946848,3.590508,0
73,2.89,1.33,0.13,0.02,61.0,40.1,20.9,12.6,4.9,3.9,135.8,2.34,4.9,1.4,1.46,0.85,0.86,90.9,10.4,0.87,26.0,2.03,0
65,4.68,2.13,0.2,0.06,61.5,39.1,22.4,13.2,4.6,3.7,139.4,2.33,6.0,0.6,1.03,1.2,3.22,89.5,10.2,0.85,27.1,2.57,0
44,4.603406,2.157543,0.172844,0.009425,73.6,40.5,33.1,14.5,5.1,4.0,146.3,2.35,4.2,1.0,1.16,1.25,3.62,91.860633,12.714961,1.000874,32.554017,3.452106,0
53,4.8477278,1.971421,0.163157,0.01747,70.9,41.2,29.7,17.3,5.8,4.4,140.2,2.4,4.2,1.0,1.73,1.77,3.35,93.074734,12.533349,0.96654,33.047475,3.903806,0
44,5.282167,1.924444,0.15032,0.014263,59.7,37.1,22.6,11.2,4.0,4.3,140.4,2.36,6.4,1.1,0.91,1.66,2.98,93.118798,12.728541,0.992288,32.75741,3.500309,0
73,5.045916,2.014818,0.179689,0.01221,65.0,35.8,29.2,9.3,3.0,4.3,145.0,2.34,7.6,1.0,1.17,1.22,3.36,91.670342,12.770883,1.007072,32.918,3.683694,1
44,5.518344,1.837139,0.140389,0.011307,56.8,34.7,22.1,18.4,7.4,3.8,141.4,2.19,7.7,0.8,1.950352,1.192077,2.966517,92.088986,13.000624,1.019427,33.688882,3.418911,0
51,4.950973,2.29587,0.253054,0.021583,85.2,44.5,40.7,13.0,4.0,5.7,141.0,2.74,9.56007,1.0,3.20164,1.450097,3.032636,91.512642,12.090617,0.943692,32.396507,4.786998,0


In [15]:
# 年龄有些奇怪的值
raw_data.with_columns(pl.col('年龄').cast(pl.Int64)).filter(pl.col('年龄') < 0)

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原,是否高血压
i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64


In [4]:
# 筛下奇怪的值
data = raw_data.filter(pl.col('年龄') >= 1)
data.null_count()

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原,是否高血压
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
# 算下每个特征和是否高血压的相关度
cols = data.columns[:-1]
corr = pl.DataFrame()
for col in cols:
    corr = corr.with_columns(pl.corr(data[col], data['是否高血压']))
corr

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,嗜碱性粒细胞数,总蛋白,白蛋白,球蛋白,总胆红素,直接胆红素,钾,钠,钙,尿素氮,谷草谷丙,甘油三酯,高密度脂蛋白,低密度脂蛋白,红细胞平均体积,凝血酶原时间,国际标准化比值,活化部分凝血活酶时间,纤维蛋白原
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.173126,-0.23544,0.155703,0.104577,-0.029597,0.389186,0.435219,0.036935,0.074015,-0.000422,0.144729,0.226332,0.333133,0.050992,-0.205804,0.045469,-0.025464,0.077063,-0.000579,0.067236,0.032862,0.107435,-0.043088


In [6]:
# 选一下相关度值在0.1以上的
rel_cols = []
discard_cols = []
for col in cols:
    if abs(corr[col][0]) >= 0.1:
        rel_cols.append(col)
    else:
        discard_cols.append(col)
print(rel_cols)
print(discard_cols)

['年龄', '中性粒细胞数', '淋巴细胞数', '嗜酸性粒细胞数', '总蛋白', '白蛋白', '钾', '钠', '钙', '谷草谷丙', '活化部分凝血活酶时间']
['嗜碱性粒细胞数', '球蛋白', '总胆红素', '直接胆红素', '尿素氮', '甘油三酯', '高密度脂蛋白', '低密度脂蛋白', '红细胞平均体积', '凝血酶原时间', '国际标准化比值', '纤维蛋白原']


In [7]:
# 获取x,y
raw_x = raw_data.select(rel_cols).to_pandas()
raw_y = raw_data.select('是否高血压').to_pandas()

In [8]:
raw_x

,年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,总蛋白,白蛋白,钾,钠,钙,谷草谷丙,活化部分凝血活酶时间
0,79.0,9.260000,2.140000,0.180000,67.948927,39.546489,4.100000,137.200000,2.340000,1.20000,33.946848
1,73.0,2.890000,1.330000,0.130000,61.000000,40.100000,3.900000,135.800000,2.340000,1.40000,26.000000
2,65.0,4.680000,2.130000,0.200000,61.500000,39.100000,3.700000,139.400000,2.330000,0.60000,27.100000
3,44.0,4.603406,2.157543,0.172844,73.600000,40.500000,4.000000,146.300000,2.350000,1.00000,32.554017
4,53.0,4.847728,1.971421,0.163157,70.900000,41.200000,4.400000,140.200000,2.400000,1.00000,33.047475
...,...,...,...,...,...,...,...,...,...,...,...
33282,68.0,4.857917,2.117560,0.194136,70.353715,43.506534,4.100000,143.000000,2.360000,1.09598,33.649698
33283,53.0,3.770986,2.197268,0.180861,76.000000,48.300000,4.127902,141.998793,2.419122,0.50000,33.556394
33284,51.0,4.710000,2.970000,0.780000,85.100000,46.200000,4.600000,140.000000,2.300000,0.80000,35.800000
33285,69.0,5.990000,2.310000,0.320000,63.700000,36.100000,4.200000,142.000000,2.050000,0.90000,41.500000


In [9]:
raw_y

,是否高血压
0,0
1,0
2,0
3,0
4,0
...,...
33282,0
33283,0
33284,1
33285,0


In [12]:
lm = linear_model.LinearRegression()
# x_train, x_test, y_train, y_test = tra
model = lm.fit(raw_x, raw_y)
# cross_val_score(lm, raw_x, raw_y, cv=5)
# model.score()

In [14]:
type(raw_x)

pandas.core.frame.DataFrame

In [27]:
x_pl = pl.from_pandas(raw_x)
y_pl = pl.from_pandas(raw_y)
for xcol in x_pl.columns:
    x_pl = x_pl.with_columns((pl.col(xcol) - pl.col(xcol).mean())/pl.col(xcol).std())

In [31]:
y_pl.filter(pl.col('是否高血压').is_in([1,0]).is_not())

/tmp/ipykernel_2140/3336041413.py:1: DeprecationWarning: `is_not` is deprecated. It has been renamed to `not_`.
  y_pl.filter(pl.col('是否高血压').is_in([1,0]).is_not())


是否高血压
i64


In [51]:
folds = 3
data_partitions = []
data_cnts = x_pl.shape[0]
part_size = data_cnts // folds
for f in range(folds):
    # data_partitions.append(x_pl[:part_size * f], y_pl[])
    l = f * part_size
    if f == folds-1:
        r = data_cnts
    else:
        r = (f+1) * part_size
    train_x = pl.concat([x_pl[:l], x_pl[r:]])
    train_y = pl.concat([y_pl[:l], y_pl[r:]])
    test_x = x_pl[l:r]
    test_y = y_pl[l:r]
    data_partitions.append((train_x, train_y, test_x, test_y))

33287 11095
0 0 11095
1 11095 22190
2 22190 33287


In [78]:
xc = pl.concat([data_partitions[2][0], data_partitions[2][2]]).sort(by=xc.columns)

In [79]:
xc.equals(x_pl.sort(by=xc.columns))

True

In [74]:
xc

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,总蛋白,白蛋白,钾,钠,钙,谷草谷丙,活化部分凝血活酶时间
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-3.208425,0.14092,0.702322,-0.716653,-0.936846,-0.747874,-0.244303,-0.191422,-0.567105,0.571598,1.204399
-3.208425,0.106886,0.318584,0.11869,0.89601,0.363986,-0.662101,0.281304,1.244152,0.097707,0.11942
-3.208425,-0.559455,1.009042,-0.287172,-2.351918,-0.747874,-2.212155,1.548552,-0.611816,-0.534148,0.110006
-3.208425,1.36429,-0.907638,-0.223642,-3.430069,-4.385245,1.662981,1.126136,-2.948959,1.361417,-0.399338
-3.208425,0.168386,0.57087,-0.072432,-3.308777,-3.845198,-0.662101,-0.563529,3.718775,0.571598,-0.156505
-3.208425,0.516284,0.439419,-0.788233,-3.699606,-4.782338,-1.178785,-0.141113,-1.986606,1.203453,-0.312905
-3.208362,0.223317,8.939959,0.571789,0.983211,1.329678,-0.109613,1.070831,0.969102,-0.332405,7.81243
-3.208362,1.147995,4.704295,-0.645073,0.384036,0.3327,0.371269,-0.985945,0.900454,-0.045807,-0.342884
-3.208362,-0.408393,-0.042571,0.285468,-2.850563,-2.606269,1.146296,-0.985945,0.281798,7.364041,0.235742


In [73]:
x_pl.sort(by='年龄')

年龄,中性粒细胞数,淋巴细胞数,嗜酸性粒细胞数,总蛋白,白蛋白,钾,钠,钙,谷草谷丙,活化部分凝血活酶时间
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
-3.208425,0.14092,0.702322,-0.716653,-0.936846,-0.747874,-0.244303,-0.191422,-0.567105,0.571598,1.204399
-3.208425,0.106886,0.318584,0.11869,0.89601,0.363986,-0.662101,0.281304,1.244152,0.097707,0.11942
-3.208425,0.168386,0.57087,-0.072432,-3.308777,-3.845198,-0.662101,-0.563529,3.718775,0.571598,-0.156505
-3.208425,0.516284,0.439419,-0.788233,-3.699606,-4.782338,-1.178785,-0.141113,-1.986606,1.203453,-0.312905
-3.208425,-0.559455,1.009042,-0.287172,-2.351918,-0.747874,-2.212155,1.548552,-0.611816,-0.534148,0.110006
-3.208425,1.36429,-0.907638,-0.223642,-3.430069,-4.385245,1.662981,1.126136,-2.948959,1.361417,-0.399338
-3.208362,6.371052,3.360566,0.285468,-2.540595,-1.732664,2.69635,0.154579,-2.399043,8.311823,0.438247
-3.208362,0.223317,8.939959,0.571789,0.983211,1.329678,-0.109613,1.070831,0.969102,-0.332405,7.81243
-3.208362,1.147995,4.704295,-0.645073,0.384036,0.3327,0.371269,-0.985945,0.900454,-0.045807,-0.342884
